# 0. Draft Filtering

In [ ]:
import pandas as pd

def clean_data(df):
    df = df.drop(columns=[
        ('Assay ID', 'IEDB IRI'), 
        ('Reference', 'IEDB IRI'), 
        ('Reference', 'Type'), 
        ('Reference', 'PMID'), 
        ('Reference', 'Submission ID'), 
        ('Reference', 'Authors'), 
        ('Reference', 'Journal'),
        ('Reference', 'Title'), 
        ('Epitope', 'Epitope IRI'), 
        ('Epitope', 'Starting Position'), 
        ('Epitope', 'Ending Position'), 
        ('Epitope', 'IRI'), 
        ('Epitope', 'Synonyms'), 
        ('Epitope', 'Source Molecule IRI'), 
        ('Epitope', 'Molecule Parent IRI'), 
        ('Epitope', 'Source Organism IRI'), 
        ('Epitope', 'Species IRI'), 
        ('Epitope', 'Comments'), 
        ('Related Object', 'Epitope Relation'), 
        ('Related Object', 'Object Type'), 
        ('Related Object', 'Name'), 
        ('Related Object', 'Starting Position'), 
        ('Related Object', 'Ending Position'), 
        ('Related Object', 'IRI'), 
        ('Related Object', 'Synonyms'), 
        ('Related Object', 'Source Molecule'), 
        ('Related Object', 'Source Molecule IRI'), 
        ('Related Object', 'Molecule Parent'), 
        ('Related Object', 'Molecule Parent IRI'), 
        ('Related Object', 'Source Organism'), 
        ('Related Object', 'Source Organism IRI'), 
        ('Related Object', 'Species'), 
        ('Related Object', 'Species IRI'), 
        ('Host', 'Geolocation'), 
        ('Host', 'Geolocation IRI'), 
        ('Host', 'Sex'), 
        ('Host', 'Age'), 
        ('in vivo Antigen', 'Epitope Relation'), 
        ('in vivo Antigen', 'Object Type'), 
        ('in vivo Antigen', 'Name'), 
        ('in vivo Antigen', 'Reference Name'), 
        ('in vivo Antigen', 'Starting Position'), 
        ('in vivo Antigen', 'Ending Position'), 
        ('in vivo Antigen', 'IRI'), 
        ('in vivo Antigen', 'Source Molecule'), 
        ('in vivo Antigen', 'Source Molecule IRI'), 
        ('in vivo Antigen', 'Molecule Parent'), 
        ('in vivo Antigen', 'Molecule Parent IRI'), 
        ('in vivo Antigen', 'Source Organism'), 
        ('in vivo Antigen', 'Source Organism IRI'), 
        ('in vivo Antigen', 'Species'), 
        ('in vivo Antigen', 'Species IRI'), 
        ('in vivo Antigen', 'Adjuvants'), 
        ('in vivo Antigen', 'Route'), 
        ('in vivo Antigen', 'Dose Schedule'), 
        ('In vitro Process', 'Process Type'), 
        ('In vitro Process', 'Epitope Relation'), 
        ('In vitro Process', 'Object Type'), 
        ('In vitro Process', 'Name'), 
        ('In vitro Process', 'Reference Name'), 
        ('In vitro Process', 'Starting Position'), 
        ('In vitro Process', 'Ending Position'), 
        ('In vitro Process', 'IRI'), 
        ('In vitro Process', 'Source Molecule'), 
        ('In vitro Process', 'Source Molecule IRI'), 
        ('In vitro Process', 'Molecule Parent'), 
        ('In vitro Process', 'Molecule Parent IRI'), 
        ('In vitro Process', 'Source Organism'), 
        ('In vitro Process', 'Source Organism IRI'), 
        ('In vitro Process', 'Species'), 
        ('In vitro Process', 'Species IRI'), 
        ('MHC Restriction', 'IRI'), 
        ('MHC Restriction', 'Evidence IRI'), 
        ('Host', 'IRI'), 
        ('in vivo Process', 'Disease IRI'), 
        ('Assay', 'IRI'), 
        ('Antigen Presenting Cell', 'Source Tissue IRI'), 
        ('Antigen Presenting Cell', 'IRI'),
        ####
        ('Antigen Presenting Cell', 'Culture Condition'),
        ('MHC Restriction', 'Evidence Code'),
        ('Assay', 'Comments'),
        ('Assay', 'Location of Assay Data in Reference'),
        ('Assay', 'PDB ID'),
        ('Antigen Processing', 'Comments'),
        ('Epitope', 'Reference Name'),
        ])

    df = df[df[('Host', 'Name')] == "Homo sapiens (human)"]
    df = df.drop(columns=[('Host', 'Name')])
    df = df[(df[('MHC Restriction', 'Name')].str.contains("*", regex=False, na=False, case=False)) & (df[('MHC Restriction', 'Name')].str.startswith("HLA", na=False)) & (~df[('MHC Restriction', 'Name')].str.contains("mutant", regex=False, na=False, case=False))]
    
    df = df[~df[('Epitope', 'Name')].str.contains("+", regex=False, na=False, case=False)]
    df = df[df[('Epitope', 'Object Type')] == "Linear peptide"]
    df = df.drop(columns=[
        ('Epitope', 'Object Type'), 
        ('Epitope', 'Modified residues'), 
        ('Epitope', 'Modifications')])
    df = df[df[('MHC Restriction', 'Class')] == "II"]
    df = df.drop(columns=[('MHC Restriction', 'Class')])
    df = df[df[('Epitope', 'Name')].str.len() == 15]
    
    return df

df = pd.read_csv(r'/home/mink/project/250228/hwj/mhc_ligand_full.csv', header=[0 , 1], low_memory=False)
df_clean = clean_data(df.copy())
df_clean.to_csv(r'draft.csv', index=False)

# 1. Binary

In [1]:
import pandas as pd

def clean_data(df):
    df_clean = pd.DataFrame()
    df_clean['Epi_Seq'] = df[('Epitope', 'Name')]
    df_clean['HLA_Name_full'] = df[('MHC Restriction', 'Name')]
    df_clean['HLA_Name'] = df_clean['HLA_Name_full'].apply(lambda x: 'HLA-'+x.split('/')[-1] if '/' in x else x)
    df_clean['Target'] = df[('Assay', 'Qualitative Measurement')].apply(lambda x: 0 if str(x) == 'Negative' else 1).values
    df_clean = df_clean.groupby(['Epi_Seq', 'HLA_Name', 'HLA_Name_full'])['Target'].max().reset_index()
    return df_clean

df = pd.read_csv(r'/home/mink/project/250303/filtered1.csv', header=[0 , 1], low_memory=False)
df_full = clean_data(df.copy())
df_full

,Epi_Seq,HLA_Name,HLA_Name_full,Target
0,AAAAAAAAAAAAAAA,HLA-DPB1*03:01,HLA-DPA1*01:03/DPB1*03:01,1
1,AAAAAAAAAAAAAAA,HLA-DPB1*06:01,HLA-DPA1*01:03/DPB1*06:01,1
2,AAAAAAAAAAAAAAA,HLA-DQB1*03:01,HLA-DQA1*05:05/DQB1*03:01,1
3,AAAAAAAAAAAAAAA,HLA-DQB1*03:02,HLA-DQA1*03:01/DQB1*03:02,1
4,AAAAAAAAAAAAAAA,HLA-DQB1*06:02,HLA-DQA1*01:02/DQB1*06:02,1
...,...,...,...,...
163979,YYVGYLQPRTFLLKY,HLA-DRB1*15:01,HLA-DRB1*15:01,0
163980,YYVWKSYVHVVDGCN,HLA-DRB1*04:04,HLA-DRB1*04:04,0
163981,YYVWKSYVHVVDGCN,HLA-DRB1*07:01,HLA-DRB1*07:01,0
163982,YYVWKSYVHVVDGCN,HLA-DRB1*15:01,HLA-DRB1*15:01,0


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = df_full
data['stratify_col'] = data['HLA_Name_full'].astype(str) + '_' + data['Target'].astype(str)

class_counts = data['stratify_col'].value_counts()
valid_classes = class_counts[class_counts >= 2].index
stratify_data = data[data['stratify_col'].isin(valid_classes)]
non_stratify_data = data[~data['stratify_col'].isin(valid_classes)]

train_strat, test_strat = train_test_split(
    stratify_data, 
    test_size=0.2, 
    stratify=stratify_data['stratify_col'], 
    random_state=100
)

train_non_strat, test_non_strat = train_test_split(
    non_stratify_data, 
    test_size=0.2, 
    random_state=100
)

train = pd.concat([train_strat, train_non_strat])
test = pd.concat([test_strat, test_non_strat])
train = train.drop(columns=['stratify_col'])
test = test.drop(columns=['stratify_col'])
train_sorted = train.sort_values(by=['Epi_Seq', 'HLA_Name', 'Target', 'HLA_Name_full'], ascending=[True, True, False, True])
test_sorted = test.sort_values(by=['Epi_Seq', 'HLA_Name', 'Target', 'HLA_Name_full'], ascending=[True, True, False, True])

print(train.shape, test.shape)

train.to_csv(r'train_non_div.csv', index=False)
test.to_csv(r'test_non_div.csv', index=False)

(131187, 4) (32797, 4)


In [ ]:
import pandas as pd
train = pd.read_csv(r'/home/alpha/project/IMG/data/250303/train_non_div.csv')
test = pd.read_csv(r'/home/alpha/project/IMG/data/250303/test_non_div.csv')

train_sorted = train.sort_values(by=['Epi_Seq', 'HLA_Name', 'Target', 'HLA_Name_full'], ascending=[True, True, False, True])
train_dedup = train_sorted.drop_duplicates(subset=['Epi_Seq', 'HLA_Name'], keep='first')
test_sorted = test.sort_values(by=['Epi_Seq', 'HLA_Name', 'Target', 'HLA_Name_full'], ascending=[True, True, False, True])
train_sorted = train.sort_values(['Target'], ascending=False)
test_dedup = test_sorted.drop_duplicates(subset=['Epi_Seq', 'HLA_Name'], keep='first')
print(len(train_dedup), len(test_dedup))

# train_dedup = train_dedup.drop(columns=['HLA_Name_full'])
# test_dedup = test_dedup.drop(columns=['HLA_Name_full'])

train_dedup.to_csv(r'train.csv', index=False)
test_dedup.to_csv(r'test.csv', index=False)

In [19]:
import pandas as pd

train_dedup = pd.read_csv(r'/home/hwjang/project/IMG/data/250303/train.csv')
test_dedup = pd.read_csv(r'/home/hwjang/project/IMG/data/250303/test.csv')

train_dedup['type'] = 'train'
test_dedup['type'] = 'test'

train_plus_test = pd.concat([train_dedup, test_dedup])
# find duplicates
duplicates = train_plus_test[train_plus_test.duplicated(subset=['Epi_Seq', 'HLA_Name'], keep=False)]
duplicates = duplicates.sort_values(by=['Epi_Seq', 'HLA_Name', 'HLA_Name_full', 'Target'], ascending=[True, True, True, False])
drop_duplicates = duplicates.drop_duplicates(subset=['Epi_Seq', 'HLA_Name'], keep='first')

train_plus_test = train_plus_test.sort_values(by=['Epi_Seq', 'HLA_Name', 'HLA_Name_full', 'Target'], ascending=[True, True, True, False])
train_plus_test_wo_duplicates = train_plus_test.drop_duplicates(subset=['Epi_Seq', 'HLA_Name'], keep='first')
train_pure = train_plus_test_wo_duplicates[train_plus_test_wo_duplicates['type'] == 'train']
test_pure = train_plus_test_wo_duplicates[train_plus_test_wo_duplicates['type'] == 'test']

print(len(test_pure)/(len(train_pure)+len(test_pure)))
train_pure = train_pure.drop(columns=['type'])
test_pure = test_pure.drop(columns=['type'])
train_pure.to_csv(r'train.csv', index=False)
test_pure.to_csv(r'test.csv', index=False)

0.20029370248697645


# 2. IC50

In [18]:
import pandas as pd
import numpy as np

def clean_data(df):
    df = df[(df[('Assay', 'Response measured')].str.contains("IC50", regex=False, na=False, case=False)) & ((df[('Assay', 'Measurement Inequality')].isna()) | (df[('Assay', 'Measurement Inequality')] == "="))]
    df = df[~df[('Assay', 'Quantitative measurement')].isna()]
    df_clean = pd.DataFrame()
    df_clean['Epi_Seq'] = df[('Epitope', 'Name')]
    df_clean['HLA_Name_full'] = df[('MHC Restriction', 'Name')]
    df_clean['HLA_Name'] = df_clean['HLA_Name_full'].apply(lambda x: 'HLA-'+x.split('/')[-1] if '/' in x else x)
    df_clean['IC50'] = pd.to_numeric(df[('Assay', 'Quantitative measurement')], errors='coerce')
    df_clean['log_IC50'] = np.log10(df_clean['IC50'])
    df_clean = df_clean.groupby(['Epi_Seq', 'HLA_Name'])['log_IC50'].mean().reset_index()
    df_clean['Target'] = (df_clean['log_IC50'] < np.log10(500)).astype(int)
    return df_clean

df = pd.read_csv(r'/home/mink/project/250303/filtered1.csv', header=[0 , 1], low_memory=False)
df_ic50 = clean_data(df.copy())
# df_ic50
# df_clean.to_csv(r'ic50.csv', index=False)

PermissionError: [Errno 13] Permission denied: '/home/mink/project/250303/filtered1.csv'

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = df_ic50
data['Target_bin'] = pd.qcut(data['Target'], q=100, duplicates='drop')
data['stratify_col'] = data['HLA_Name'].astype(str) + '_' + data['Target_bin'].astype(str)

class_counts = data['stratify_col'].value_counts()
valid_classes = class_counts[class_counts >= 2].index
stratify_data = data[data['stratify_col'].isin(valid_classes)]
non_stratify_data = data[~data['stratify_col'].isin(valid_classes)]

train_strat, test_strat = train_test_split(
    stratify_data, 
    test_size=0.2, 
    stratify=stratify_data['stratify_col'], 
    random_state=100
)

train_non_strat, test_non_strat = train_test_split(
    non_stratify_data, 
    test_size=0.2, 
    random_state=100
)

train = pd.concat([train_strat, train_non_strat])
test = pd.concat([test_strat, test_non_strat])
train = train.drop(columns=['Target_bin', 'stratify_col'])
test = test.drop(columns=['Target_bin', 'stratify_col'])

print(train.shape, test.shape)
train.to_csv(r'train_ic50.csv', index=False)
test.to_csv(r'test_ic50.csv', index=False)

(34019, 4) (8506, 4)
